In [3]:
import os
import pickle
import sys

import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import OneHotEncoder

sys.path.append('..')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src.model_utils import downsample, get_unique_values, reshape_data

## Paths to data

In [13]:
# defining paths
path_to_npys_data = os.path.join("..", "data", "npys_data")

pathTarget_3 = os.path.join(os.path.join(path_to_npys_data, "target_croplands.npy"))
pathFeatures = os.path.join(path_to_npys_data, "features_initial_data.npy")
pathMorf = os.path.join(path_to_npys_data, "features_morf_data.npy")

##  Data

In [14]:
# Features
X = pd.DataFrame.from_dict(np.load(pathFeatures, allow_pickle=True), orient="columns")
morf = pd.DataFrame.from_dict(
        np.load(pathMorf, allow_pickle=True), orient="columns"
    )
X = pd.concat([X, morf], axis=1)
keys = list(X.keys())

with open(os.path.join(path_to_npys_data, "keys.pkl"), 'wb') as file:
    # Dump the list into the file
    pickle.dump(keys, file)
    
# Target Variable
y = pd.DataFrame.from_dict(np.load(pathTarget_3, allow_pickle=True), orient="columns")
y = y["Target"].astype(int).to_numpy()

In [15]:
# Set classes 4,5 to 0
y = np.where(y > 3, 0, y)


## Data Description

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13638352 entries, 0 to 13638351
Columns: 162 entries, sfcWindmax_M1 to morf_33_10
dtypes: float64(162)
memory usage: 16.5 GB


In [10]:
X.head()

,sfcWindmax_M1,sfcWindmax_M2,sfcWindmax_M3,sfcWindmax_M4,sfcWindmax_M5,sfcWindmax_M6,sfcWindmax_M7,sfcWindmax_M8,sfcWindmax_M9,sfcWindmax_M10,...,morf_33_1,morf_33_2,morf_33_3,morf_33_4,morf_33_5,morf_33_6,morf_33_7,morf_33_8,morf_33_9,morf_33_10
0,1.3,1.1,1.0,1.5,1.8,1.2,1.8,1.3,2.0,1.2,...,0.500770,177.346313,0.707366,0.000439,0.207605,0.062292,-0.257171,-0.257601,0.062723,0.713253
1,1.3,1.1,1.0,1.5,1.8,1.2,1.8,1.3,2.0,1.2,...,0.505880,172.595993,0.707884,0.000409,0.201472,0.057969,-0.252120,-0.256513,0.062362,0.713270
2,1.3,1.1,1.0,1.5,1.8,1.2,1.8,1.3,2.0,1.2,...,0.554733,164.047455,0.708955,0.000276,0.169685,0.039095,-0.232850,-0.254409,0.060654,0.713656
3,1.3,1.1,1.0,1.5,1.8,1.2,1.8,1.3,2.0,1.2,...,0.630440,159.827286,0.709747,0.000169,0.140592,0.023969,-0.219259,-0.253394,0.058104,0.714367
4,1.3,1.1,1.0,1.5,1.8,1.2,1.8,1.3,2.0,1.2,...,0.731494,159.067596,0.710274,0.000134,0.119920,0.019065,-0.216999,-0.253654,0.055720,0.715481


In [11]:
X.describe()

,sfcWindmax_M1,sfcWindmax_M2,sfcWindmax_M3,sfcWindmax_M4,sfcWindmax_M5,sfcWindmax_M6,sfcWindmax_M7,sfcWindmax_M8,sfcWindmax_M9,sfcWindmax_M10,...,morf_33_1,morf_33_2,morf_33_3,morf_33_4,morf_33_5,morf_33_6,morf_33_7,morf_33_8,morf_33_9,morf_33_10
count,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,...,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07,1.363835e+07
mean,1.380395e+00,1.434316e+00,1.900162e+00,1.415603e+00,1.612003e+00,1.577536e+00,1.505585e+00,1.617272e+00,1.437621e+00,1.418018e+00,...,1.276664e+00,1.813717e+02,7.065281e-01,-1.145581e-04,1.603969e-01,-1.585194e-02,-4.116076e-01,-8.372061e-01,3.720454e-01,7.065811e-01
std,4.019296e-01,4.296952e-01,4.451495e-01,3.411769e-01,4.015478e-01,3.929840e-01,3.429174e-01,3.614533e-01,4.322675e-01,3.602904e-01,...,2.311596e+00,1.139481e+02,8.674426e-03,3.606118e-03,5.425503e-01,5.146567e-01,1.275188e+00,9.456932e-01,9.990941e-01,3.116058e-02
min,0.000000e+00,1.000000e-01,3.000000e-01,3.000000e-01,4.000000e-01,2.000000e-01,3.000000e-01,2.000000e-01,2.000000e-01,3.000000e-01,...,0.000000e+00,2.666816e-06,4.865897e-01,-6.034555e-02,-2.363145e+01,-8.565227e+00,-9.004397e+00,-9.081969e+00,-3.625468e-02,2.555841e-01
25%,1.100000e+00,1.100000e+00,1.600000e+00,1.200000e+00,1.300000e+00,1.300000e+00,1.300000e+00,1.400000e+00,1.100000e+00,1.200000e+00,...,1.882204e-01,9.546693e+01,7.054717e-01,-3.590560e-05,0.000000e+00,-5.092960e-03,-7.440413e-01,-1.168685e+00,5.363546e-02,7.020443e-01
50%,1.400000e+00,1.400000e+00,1.900000e+00,1.400000e+00,1.600000e+00,1.600000e+00,1.500000e+00,1.600000e+00,1.400000e+00,1.400000e+00,...,4.743695e-01,1.800558e+02,7.070554e-01,0.000000e+00,1.296582e-01,0.000000e+00,-2.179654e-01,-4.492523e-01,1.203083e-01,7.070925e-01
75%,1.600000e+00,1.700000e+00,2.200000e+00,1.600000e+00,1.900000e+00,1.800000e+00,1.700000e+00,1.900000e+00,1.700000e+00,1.700000e+00,...,1.316145e+00,2.768814e+02,7.083952e-01,7.287971e-04,3.155357e-01,1.034390e-01,0.000000e+00,-2.202093e-01,3.204067e-01,7.118509e-01
max,3.000000e+00,2.900000e+00,3.600000e+00,3.000000e+00,3.100000e+00,3.200000e+00,2.900000e+00,3.000000e+00,3.000000e+00,2.900000e+00,...,3.149512e+01,3.600000e+02,8.591240e-01,1.057680e-01,1.099178e+01,1.499100e+01,1.493593e+01,6.169987e-02,1.504727e+01,9.723462e-01


## Data Preparation 


### Target one hot encoding and Train/test split

In [16]:
y = pd.DataFrame(y, columns=["Target"])
# read data and apply one-hot encoding
ohe = OneHotEncoder(handle_unknown="ignore", sparse=False).fit(y)
y = ohe.transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=123
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.15, stratify=y_test, random_state=123
)

# Define scaler based on whole dataset
scaler = MinMaxScaler()
minmax = scaler.fit(X_train)
joblib.dump(minmax, os.path.join(path_to_npys_data, "scaler.save"))

# Normalization using minmax scaler
X_train = minmax.transform(X_train)
X_test = minmax.transform(X_test)
X_val = minmax.transform(X_val)

X = dict()
y = dict()

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("X_val shape:", X_val.shape)

X_train shape: (10228764, 162)
X_test shape: (2898149, 162)
X_val shape: (511439, 162)


In [18]:
# Check Classes distribution
get_unique_values(np.argmax(y_train, 1))

[(1, 110024), (2, 439606), (3, 879902), (0, 8799232)]

In [19]:
# Downsampling Class 0 up to Class 2 and oversampling Class 1
X_train, y_train = downsample(X_train, np.argmax(y_train, 1), oversampling=False)
X_val, y_val = downsample(X_val, np.argmax(y_val, 1), oversampling=False)

Initial data:
[(0, 8799232), (3, 879902), (2, 439606), (1, 110024)]
Resampled data:
[(0, 879902), (3, 879902), (2, 439606), (1, 110024)]
Initial data:
[(0, 439962), (3, 43995), (2, 21981), (1, 5501)]
Resampled data:
[(0, 43995), (3, 43995), (2, 21981), (1, 5501)]


In [20]:
X["Train"] = X_train
X["Val"] = X_val
X["Test"] = X_test
y["Train"] = ohe.transform(pd.DataFrame(y_train))
y["Val"] = ohe.transform(pd.DataFrame(y_val))
y["Test"] = y_test

In [21]:
# save dictionary pkl file
with open(os.path.join("..", "data", "processed_files", "pkls", "X_down.pkl"), "wb") as fp:
    pickle.dump(X, fp)

with open(os.path.join("..", "data", "processed_files", "pkls", "y_down.pkl"), "wb") as fp:
    pickle.dump(y, fp)

In [22]:
X["Train"] = reshape_data(pd.DataFrame(X_train, columns=keys))
X["Val"] = reshape_data(pd.DataFrame(X_val, columns=keys))
X["Test"] = reshape_data(pd.DataFrame(X_test, columns=keys))

In [24]:
# save dictionary pkl file

with open(os.path.join("..", "data", "processed_files", "pkls", "X_down_lstm.pkl"), "wb") as fp:
    pickle.dump(X, fp)

with open(os.path.join("..", "data", "processed_files", "pkls", "y_down_lstm.pkl"), "wb") as fp:
    pickle.dump(y, fp)